In [1]:
import sys
import os

In [19]:
sys.path.append(os.path.abspath('../../../../src'))

from read_and_write_docs import read_jsonl, read_rds
from tokenize_and_score import load_model
from utils import apply_temp_doc_id, build_metadata_df
from n_gram_functions import (
    common_ngrams,
    pretty_print_common_ngrams,
    keep_before_phrase,
    score_phrases,
    add_pmf_column,
    score_phrases_no_context
)

In [7]:
tokenizer, model = load_model("/Volumes/BCross/models/Qwen 2.5/Qwen2.5-0.5B-Instruct")

In [3]:
corpus = "Wiki"
data_type = "training"

known_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
known = read_jsonl(known_loc)
known = apply_temp_doc_id(known)

unknown_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
unknown = read_jsonl(unknown_loc)
unknown_df = apply_temp_doc_id(unknown)

metadata_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/metadata.rds"
metadata = read_rds(metadata_loc)
filtered_metadata = metadata[metadata['corpus'] == corpus]
agg_metadata = build_metadata_df(filtered_metadata, known, unknown)

In [4]:
problem_dataset_agg = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_agg.jsonl")
problem_dataset_profile = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_profile.jsonl")

In [24]:
diff_author_problems = problem_dataset_agg[problem_dataset_agg['known_author'] != problem_dataset_agg['unknown_author']].copy()
diff_author_problems.sort_values(["highest_common_count"], ascending=[False], inplace=True)
diff_author_problems[diff_author_problems['highest_common_count'] >= 3].head(50)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
287,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_3,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
161,Caboga vs Chanakyathegreat,Caboga,Chanakyathegreat,caboga_text_5,chanakyathegreat_text_1,5,Ġ 2 0 0 8
262,Dweller vs Ecelan,Dweller,Ecelan,dweller_text_3,ecelan_text_2,5,.Ċ i Ġdon 't Ġthink
364,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_12,garda40_text_1,5,Ġ 2 0 1 1
365,Fyunck(click) vs Garda40,Fyunck(click),Garda40,fyunck_click_text_2,garda40_text_1,5,Ġ 2 0 1 2
395,Gwen_Gale vs Habap,Gwen_Gale,Habap,gwen_gale_text_3,habap_text_1,5,", Ġbut Ġi Ġdon 't"
286,Enemesis vs Equanimous1,Enemesis,Equanimous1,enemesis_text_2,equanimous1_text_5,5,Ġthe Ġcurrent Ġstate Ġof Ġthe
441,HeadleyDown vs Hipocrite,HeadleyDown,Hipocrite,headleydown_text_2,hipocrite_text_4,5,Ġon Ġthis Ġtalk Ġpage .Ċ
135,Britmax vs BrotherDarksoul,Britmax,BrotherDarksoul,britmax_text_1,brotherdarksoul_text_4,4,Ġis Ġno Ġneed Ġto
165,Chanakyathegreat vs Cla68,Chanakyathegreat,Cla68,chanakyathegreat_text_10,cla68_text_1,4,", Ġi Ġdon 't"


In [15]:
known_doc = "enemesis_text_2"
known_text = known[known['doc_id'] == known_doc].reset_index().loc[0, 'text']

unknown_doc = "equanimous1_text_5"
unknown_text = unknown[unknown['doc_id'] == unknown_doc].reset_index().loc[0, 'text']

In [16]:
common = common_ngrams(known_text, unknown_text, 2, model, tokenizer)
pretty_print_common_ngrams(common, tokenizer=tokenizer, order='len_asc')

2-grams (15): [' an agenda', ' an article', ' from the', ' in a', ' in the', ' information is', ' is not', ' this article', ' to be', ' to the', ' which is', ' you can', ' you have', '.\nthe', '.\nthey']
3-grams (2): [' the article and', '.\nthere is']
5-grams (1): [' the current state of the']


In [17]:
print(f"""
<DOC>
{known_text}
</DOC>
<NGRAM>
""")


<DOC>
The format is wrong it is not descriptive.
The information is used to create judgement.
Now you have a COI with this article can I ask and instruct you nicely to reformat it to something that is informative and educational as well descriptive of your views?
Please write it in a non pov style to satisfy the needs of the reader.
They will want to be involved in clarifying the article and naturally so.
He to me appears to have an agenda that is beyond writing an article for artistic creativity, the need to present the facts and or present an interesting article.
In other words the reasoning is not pure it is driven by gain in the real world and then a loss of knowledge about NLP to the patrons of wikipedia which is many and varied.
GoodDay, I thought you had no interest in this article.
Snowded made a comment mentioning spas and meat puppets that claim is easily defendable by saying the truth.
There is an absolutely huge community of NLP practitioners you can expect with the curren

## Phrase 1 -  " the current state of the"

In [20]:
p_1 = " the current state of the"

para_1 = [" the present state of the", " the existing state of the", " the contemporary state of the", " the present-day state of the", " the current status of the", " the present status of the", " the existing status of the", " the contemporary status of the", " the present-day status of the", " the current condition of the", " the present condition of the", " the existing condition of the", " the contemporary condition of the", " the present-day condition of the", " the current situation of the", " the present situation of the", " the existing situation of the", " the contemporary situation of the", " the present-day situation of the", " the current shape of the", " the present shape of the", " the existing shape of the", " the contemporary shape of the", " the present-day shape of the", " the current form of the", " the present form of the", " the existing form of the", " the contemporary form of the", " the present-day form of the", " the curent state of the", " the current state of teh", " the current state o' the", " the present state o' the", " the current status o' the", " the current condition o' the", " the current situation o' the", " the current form o' the", " the current shape o' the"]

known_base_p1 = keep_before_phrase(known_text, p_1, True)
unknown_base_p1 = keep_before_phrase(unknown_text, p_1, True)

In [21]:
known_p1_scores = score_phrases(known_base_p1, p_1, para_1, tokenizer, model)
unknown_p1_scores = score_phrases(unknown_base_p1, p_1, para_1, tokenizer, model)

→ Scoring base_text alone…
   base_total = -884.8537

→ [1/39] Processing reference…
→ [2/39] Processing paraphrase…
→ [3/39] Processing paraphrase…
→ [4/39] Processing paraphrase…
→ [5/39] Processing paraphrase…
→ [6/39] Processing paraphrase…
→ [7/39] Processing paraphrase…
→ [8/39] Processing paraphrase…
→ [9/39] Processing paraphrase…
→ [10/39] Processing paraphrase…
→ [11/39] Processing paraphrase…
→ [12/39] Processing paraphrase…
→ [13/39] Processing paraphrase…
→ [14/39] Processing paraphrase…
→ [15/39] Processing paraphrase…
→ [16/39] Processing paraphrase…
→ [17/39] Processing paraphrase…
→ [18/39] Processing paraphrase…
→ [19/39] Processing paraphrase…
→ [20/39] Processing paraphrase…
→ [21/39] Processing paraphrase…
→ [22/39] Processing paraphrase…
→ [23/39] Processing paraphrase…
→ [24/39] Processing paraphrase…
→ [25/39] Processing paraphrase…
→ [26/39] Processing paraphrase…
→ [27/39] Processing paraphrase…
→ [28/39] Processing paraphrase…
→ [29/39] Processing paraphrase…

In [22]:
known_p1_pmf = add_pmf_column(known_p1_scores, 'phrase_log_probs')
unknown_p1_pmf = add_pmf_column(unknown_p1_scores, 'phrase_log_probs')

In [23]:
p1_pmf = score_phrases_no_context(p_1, para_1, tokenizer, model)
p1_pmf = add_pmf_column(p1_pmf, 'log_probs')

→ [1/39] Processing reference…
→ [2/39] Processing paraphrase…
→ [3/39] Processing paraphrase…
→ [4/39] Processing paraphrase…
→ [5/39] Processing paraphrase…
→ [6/39] Processing paraphrase…
→ [7/39] Processing paraphrase…
→ [8/39] Processing paraphrase…
→ [9/39] Processing paraphrase…
→ [10/39] Processing paraphrase…
→ [11/39] Processing paraphrase…
→ [12/39] Processing paraphrase…
→ [13/39] Processing paraphrase…
→ [14/39] Processing paraphrase…
→ [15/39] Processing paraphrase…
→ [16/39] Processing paraphrase…
→ [17/39] Processing paraphrase…
→ [18/39] Processing paraphrase…
→ [19/39] Processing paraphrase…
→ [20/39] Processing paraphrase…
→ [21/39] Processing paraphrase…
→ [22/39] Processing paraphrase…
→ [23/39] Processing paraphrase…
→ [24/39] Processing paraphrase…
→ [25/39] Processing paraphrase…
→ [26/39] Processing paraphrase…
→ [27/39] Processing paraphrase…
→ [28/39] Processing paraphrase…
→ [29/39] Processing paraphrase…
→ [30/39] Processing paraphrase…
→ [31/39] Processing

## Phrase 2 - " the article and"

In [25]:
p_2 = " the article and"

para_2 = [" the piece and", " the entry and", " the write-up and", " the write up and", " the writeup and", " the page and", " the artical and", " the articel and", " the articl and", " the artcle and", " the article &", " the article 'n", " the article adn", " the article an'", " the piece &", " the entry &", " the write-up &", " the write up &", " the writeup &", " the page &", " the piece 'n", " the entry 'n", " the write-up 'n", " the write up 'n", " the writeup 'n", " the page 'n", " the piece adn", " the entry adn", " the write-up adn", " the write up adn", " the writeup adn", " the page adn", " the piece an'", " the entry an'", " the write-up an'", " the write up an'", " the writeup an'", " the page an'", " the artical &", " the articel &", " the articl &", " the artcle &", " the artical 'n", " the articel 'n", " the articl 'n", " the artcle 'n", " the artical adn", " the articel adn", " the articl adn", " the artcle adn", " the artical an'", " the articel an'", " the articl an'", " the artcle an'"]

known_base_p2 = keep_before_phrase(known_text, p_2, True)
unknown_base_p2 = keep_before_phrase(unknown_text, p_2, True)

In [26]:
known_p2_scores = score_phrases(known_base_p2, p_2, para_2, tokenizer, model)
unknown_p2_scores = score_phrases(unknown_base_p2, p_2, para_2, tokenizer, model)

→ Scoring base_text alone…
   base_total = -334.8739

→ [1/55] Processing reference…
→ [2/55] Processing paraphrase…
→ [3/55] Processing paraphrase…
→ [4/55] Processing paraphrase…
→ [5/55] Processing paraphrase…
→ [6/55] Processing paraphrase…
→ [7/55] Processing paraphrase…
→ [8/55] Processing paraphrase…
→ [9/55] Processing paraphrase…
→ [10/55] Processing paraphrase…
→ [11/55] Processing paraphrase…
→ [12/55] Processing paraphrase…
→ [13/55] Processing paraphrase…
→ [14/55] Processing paraphrase…
→ [15/55] Processing paraphrase…
→ [16/55] Processing paraphrase…
→ [17/55] Processing paraphrase…
→ [18/55] Processing paraphrase…
→ [19/55] Processing paraphrase…
→ [20/55] Processing paraphrase…
→ [21/55] Processing paraphrase…
→ [22/55] Processing paraphrase…
→ [23/55] Processing paraphrase…
→ [24/55] Processing paraphrase…
→ [25/55] Processing paraphrase…
→ [26/55] Processing paraphrase…
→ [27/55] Processing paraphrase…
→ [28/55] Processing paraphrase…
→ [29/55] Processing paraphrase…

In [27]:
known_p2_pmf = add_pmf_column(known_p2_scores, 'phrase_log_probs')
unknown_p2_pmf = add_pmf_column(unknown_p2_scores, 'phrase_log_probs')

In [28]:
p2_pmf = score_phrases_no_context(p_2, para_2, tokenizer, model)
p2_pmf = add_pmf_column(p2_pmf, 'log_probs')

→ [1/55] Processing reference…
→ [2/55] Processing paraphrase…
→ [3/55] Processing paraphrase…
→ [4/55] Processing paraphrase…
→ [5/55] Processing paraphrase…
→ [6/55] Processing paraphrase…
→ [7/55] Processing paraphrase…
→ [8/55] Processing paraphrase…
→ [9/55] Processing paraphrase…
→ [10/55] Processing paraphrase…
→ [11/55] Processing paraphrase…
→ [12/55] Processing paraphrase…
→ [13/55] Processing paraphrase…
→ [14/55] Processing paraphrase…
→ [15/55] Processing paraphrase…
→ [16/55] Processing paraphrase…
→ [17/55] Processing paraphrase…
→ [18/55] Processing paraphrase…
→ [19/55] Processing paraphrase…
→ [20/55] Processing paraphrase…
→ [21/55] Processing paraphrase…
→ [22/55] Processing paraphrase…
→ [23/55] Processing paraphrase…
→ [24/55] Processing paraphrase…
→ [25/55] Processing paraphrase…
→ [26/55] Processing paraphrase…
→ [27/55] Processing paraphrase…
→ [28/55] Processing paraphrase…
→ [29/55] Processing paraphrase…
→ [30/55] Processing paraphrase…
→ [31/55] Processing